### Evaluation of sensor data at A034

---

### Purpose

The purpose of this file is to evaluate real sensor data from junction A034 in Darmstadt. The city of Darmstadt employs various technologies to monitor traffic at different locations throughout the city. Similarly, the simulation software SUMO provides the capability to simulate sensor data.

This feature is particularly useful when aiming to align simulated traffic with real-world data. By comparing the data from simulated sensors with the actual sensor readings, adjustments can be made to the simulation until the traffic patterns closely match the real-world measurements.

---

### Idea

The approach to achieving the purpose of this file begins by extracting all the necessary data into a DataFrame. Upon reviewing the raw data, it becomes clear that not all the information in the CSV file is required. Therefore, the first step is to filter out only the relevant data from the CSV file or trim the DataFrame until only the essential information remains.

Next, a function is implemented to aggregate the number of vehicles detected by the sensor based on a specified time interval. Finally, the results are not only presented as a DataFrame but also visualized in forms such as line graphs or heatmaps for better interpretation and analysis.

---

### Code

In [32]:
#alle notewendigen Bibliotheken einbinden
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from typing import List


---

Unfortunatly the provided csv-files are not completely consistant, sometimes some rows look like this: 

>A034;18.09.2023 09:57:00;18.09.2023 11:57:00;60000;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
>
>A034;18.09.2023 09:58:00;18.09.2023 11:58:00;60000;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
>
>A034;18.09.2023 09:59:00;18.09.2023 11:59:00;60000;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

In this case, the sensors that should provide data between the semicolons either malfunctioned or another issue caused the absence of data. Ideally, we would assume that these missing values should be recorded as 0, since the sensors did not provide any data. Additionally, our function cannot handle **"not a number" (NaN)** inputs properly. Therefore, we require another function to fill the empty spaces between the semicolons with 0. The function **zero_between_semicolons** is designed specifically to address this issue.

In [33]:
def zero_between_semikolons(inputfile, outputfile,replace,counter):
    
    # Datei einlesen
    with open(inputfile, "r") as file:
        data = file.read()

# Ersetzen von ";;" durch ";0;"
    data_modified = re.sub(r";;", replace, data)
    data_modified = re.sub(r";\n","\n",data_modified)

# In eine neue Datei schreiben
    with open(outputfile, "w") as file:
        file.write(data_modified)
    while(counter < 2):
        counter = counter +1
        zero_between_semikolons(outputfile,outputfile,replace,counter)
        

---

The function create_data_analysis is designed to process, analyze, and visualize traffic data collected by multiple sensors. This data is typically stored in a CSV file with raw readings from sensors, which may include missing values or formatting issues. The function handles these challenges, aggregates the data into defined time intervals, and provides insights through visualizations.
Key Features:

1. Data Cleaning:
- Handles missing values in the CSV file by replacing empty or invalid entries with zero using a helper function zero_between_semicolons.
- Renames columns for consistency and ease of analysis.

2. Time-Based Aggregation:
- Groups sensor data into user-defined time intervals (e.g., 5, 10, or 30 minutes) to better analyze trends over time.
- Uses floor operations to round timestamps to the nearest interval.

3. Data Transformation:
- Converts the aggregated data from a "wide" format (with sensors as columns) to a "long" format, making it more suitable for advanced visualizations and analysis.
- Adds placeholder columns for various vehicle types and speeds, enabling further extension or integration with additional datasets.

4. Visualization:
- Generates a heatmap to display traffic intensity (qPKW) for each sensor over time, highlighting patterns and anomalies.
- Creates a scatterplot and line plot to visualize average traffic values (qPKW) across sensors, enabling comparisons between sensor performance.

5. Output:
- Saves the cleaned and processed data as a tab-separated file for further use.
- Provides interactive visual insights to assist in traffic analysis and simulation adjustments.

In [34]:
def create_data_analysis(csv_files: List[str], output_file, intervall_in_min, detector_name):
    #Zeitintervall vorbereiten für dt.floor vorbereiten
    time_interval = str(intervall_in_min) + "T"
    #Erstellen des Spaltennamens fürs Zeitintervall
    time_colum_name = "Time Intervall : " + str(intervall_in_min) + " min"

    #CSV-Datei vorbereiten falls NAN oder semikolons am ende Problematik von A034_20230401_000000_-_20230411_100000_1min.csv
    corrected_csv_files = []
    for i in csv_files:
        corrected_csv_file = "C" + i
        zero_between_semikolons(i, corrected_csv_file,";0;",0)
        corrected_csv_files.append(corrected_csv_file)

    #Spaltennamen erhalten
    for i in corrected_csv_files:
        A034 = pd.read_csv(i,sep=";")
        spalten = A034.columns #Columns gibt die Spaltennamen von A034 zurück
        #Aus Spalten werden nur die spalten herausgesucht, welche "(Belegungen/Intervall)" im Namen haben
        belegungen_columns = [col for col in spalten if '(Belegungen/Intervall)' in col]
        #Falls es Spalten gibt mit dem Namen "Intervallbeginn (Lokalzeit)", werden diese an den Anfang von belegungen_columns gestellt
        if "Intervallbeginn (Lokalzeit)" in spalten:
            belegungen_columns.insert(0, "Intervallbeginn (Lokalzeit)")


        # Automatische Umwandlung
        converted_columns = [f"{detector_name}_{col.split(' ')[0]}" for col in belegungen_columns] #sorgt dafür das aus dem spaltennamen: Sensor1 (Belegung/Intervall) der Spaltenname A034_Sensor1 wird
        
        for i in range(len(belegungen_columns)):
            A034 = A034.rename(columns={belegungen_columns[i]:converted_columns[i]}) #benennt die spalten belegungen_columns entsprechend converted_columns
        A034[converted_columns[0]] = pd.to_datetime(A034[converted_columns[0]],format="%d.%m.%Y %H:%M:%S") # convertiert die erste spalte in das format datetime
        A034[time_colum_name] = A034[converted_columns[0]].dt.floor(time_interval)  # Zeit auf 5-Minuten-Intervalle runden
    
        result = A034.groupby(time_colum_name)[converted_columns[1:]].sum().reset_index() #gruppiert alle gleichen inhalte der spalte "time_colum_name" und summiert die inhalte der übrigen spalten auf

        start_time = result[time_colum_name].min() # speichert den kleinsten wert der Spalte time_colum_name als start_time
        result['Time'] = (result[time_colum_name] - start_time).dt.total_seconds() / 60 # zieht für jeden Wert die start_time ab und berechnet so die Minuten ausgehend vom begin
        result = result.drop(columns=[time_colum_name], errors="ignore") 
        
            
        #Erreicht die gewünschte Aussgabe
        #um von wide nach long zu transformieren (alle spalten auser time zu zeilen wandeln)
        melted = pd.melt(
            result, 
            #id_vars=[time_colum_name, 'Time'],  # Version mit Globaler zeit 
            id_vars=['Time'],  # Zeit bleibt als Spalte # Version ohne Globale Zeit
            var_name="Detector", # Name für die neuen Zeilen
            value_name="qPKW",         # Name der Werte

        )
        melted.set_index(['Time', "Detector"], inplace=True)
        melted.sort_index(inplace=True)
        melted["qFG"] = 0
        melted["qRF"] = 0
        melted["qBus"] = 0
        melted["qLKW"] = 0
        melted["vPKW"] = 0
        melted["vFG"] = 0
        melted["vRF"] = 0
        melted["vBus"] = 0
        melted["vLKW"] = 0
        
        melted = melted.swaplevel("Detector",'Time')
        melted = melted.sort_index(level=['Time', 'Detector'], ascending=[True, True])
        melted = melted.reset_index()

        #Daten in txt-Datei speichern
        #melted.to_csv(output_file, sep="\t", index=False)
        #melted.to_csv("SemikolonSepperated_" + output_file , sep=";",index = False, header= False )
        melted.to_csv("SemikolonSepperated_" + output_file , sep=";",index = False)
        return melted
        # Learnings: 
        # Relevant waren das der Header "Detector", "Time", und "qPKW" enthält und das die Datei nicht durch
        # Tabulatoren sondern durch Semikolons getrennt wurde
    

To-Do:
- mehrere Files übergeben 
- erstes File gibt header an
- restlichen Files werden entsprechend der Zeit und dem Sensornamen einsortiert


In [35]:
files = ["A034_20230101_000000_-_20230201_000000_1min.csv", "A036_20230101_000000_-_20230201_000000_1min.csv"]
create_data_analysis(files,"output3.txt",10,"A036")

,Detector,Time,qPKW,qFG,qRF,qBus,qLKW,vPKW,vFG,vRF,vBus,vLKW
0,A036_D41,0.0,0,0,0,0,0,0,0,0,0,0
1,A036_D42,0.0,0,0,0,0,0,0,0,0,0,0
2,A036_D43,0.0,5,0,0,0,0,0,0,0,0,0
3,A036_D44,0.0,6,0,0,0,0,0,0,0,0,0
4,A036_D81,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
133915,A036_V45_Anf,44630.0,10,0,0,0,0,0,0,0,0,0
133916,A036_V46_Anf,44630.0,7,0,0,0,0,0,0,0,0,0
133917,A036_V85_Anf,44630.0,8,0,0,0,0,0,0,0,0,0
133918,A036_V86_Anf,44630.0,6,0,0,0,0,0,0,0,0,0
